# TP 2 : récursivité et conditions

Dans tout le TP, il est interdit d'utiliser des boucles (pas de `for` ni `while`).

## Petites questions

1. Définir une fonction `divise` telle que `divise a b` renvoie `true` si  `a` divise `b`, `false` sinon. Il est interdit d'utiliser `if`.
1. Définir une fonction récursive `somme` telle que `somme n` renvoie $\sum_{k=1}^n k^2$.
2. Définir une fonction récursive `u` telle que `u n` renvoie $u_n$ définie par :
$$u_0 = 42$$
$$u_{n} = 3\sqrt{u_{n - 1}} + 2$$
3. Soient $a$ et $b$ deux entiers et $q, r$ le quotient et reste de la division euclidienne de $a$ par $b$ ($a = bq + r$). En utilisant le fait que $PGCD(a, b)$ = $PGCD(b, r)$, écrire une fonction `euclide` telle que `euclide a b` renvoie le PGCD de $a$ et $b$.

In [1]:
let divise a b =
    a mod b = 0

val divise : int -> int -> bool = <fun>


In [3]:
divise 10 4

- : bool = false


In [11]:
let rec somme n =
    if n = 1 then 1
    else n*n + somme (n-1);;
somme 4

val somme : int -> int = <fun>


- : int = 30


In [22]:
let rec u n =
    if n = 0. then 42.
    else 3. *. sqrt (u (n-.1.)) +. 2.;;
u 3.

val u : float -> float = <fun>


- : float = 13.9593276159321622


In [30]:
let rec euclide a b =
    let r = a mod b in
    if r = 0 then b
    else euclide b r

val euclide : int -> int -> int = <fun>


In [37]:
euclide 186 24

- : int = 6


## Exponentiation rapide

1. Écrire une fonction récursive `puissance` naïve (c'est à dire très simple) telle que `puissance a n` renvoie $a^n$. Combien effectue t-elle de multiplications (en fonction de n) ?
2. Écrire une fonction récursive `exp_rapide` pour calculer $a^n$, en utilisant les relations suivantes :
$$ 
\begin{cases} 
a^n = (a^{\frac{n}{2}})^2 ~~~~~~~~~\text{si }n\text{ est pair}\\
a^{n} = a \times (a^{\frac{n-1}{2}})^2 ~~~~~\text{sinon}
\end{cases}
$$
**Remarque** : On montrera plus tard que cette 2ème version demande environ $\ln(n)$ multiplications seulement.

In [39]:
let rec puissance a n =
    if n = 1 then a
    else a * puissance a (n-1)

val puissance : int -> int -> int = <fun>


In [41]:
puissance 2 10

- : int = 1024


In [27]:
let rec exp_rapide a n =
    if n mod 2 = 0 then
        (if n = 0 then 1
        else let e = exp_rapide a (n/2) in
        e*e)
    else (if n = 1 then a
        else let f = exp_rapide a ((n-1)/2) in
        f*f*a);;
exp_rapide 2 7

val exp_rapide : int -> int -> int = <fun>


- : int = 128


interrupt: intterupt

## Temps de vol de la suite de Syracuse

La suite de Syracuse d'un entier $a$ est définie par :  
$$u_0 = a$$
$$u_{n+1} =
\begin{cases} 
     \frac{u_n}{2}, \text{si } u_n \text{ est pair}\\
    3u_n + 1, \text{sinon}\\
\end{cases}$$

Le temps de vol de $(u_n)_n$ est le plus petit entier $t$ tel que $u_t = 1$.  

1. Écrire une fonction récursive `temps_vol` telle que `temps_vol a` renvoie le temps de vol de $(u_n)_n$ (où $u_0 = a$).

In [6]:
let rec temps_vol a =
    if a = 1 then 0
    else if a mod 2 = 0
        then 1+temps_vol (a/2)
        else 1+temps_vol (3*a+1);;
temps_vol 4

val temps_vol : int -> int = <fun>


- : int = 2


interrupt: intterupt

## Fonction mystérieuse

Tester la fonction suivante, conjecturer un théorème et prouvez-le par récurrence.

In [1]:
let rec f n =
    if n > 100 then n - 10 
    else f (f (n + 11)) ;;

val f : int -> int = <fun>
